In [93]:
#chat model
import getpass 
import os


if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [94]:
#embedding model
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [95]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name = "agent-model",
    embedding_function = embeddings,
    persist_directory = "./chroma_langchain_db",
)

In [96]:
import bs4
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://www.geeksforgeeks.org/linux-unix/introduction-to-linux-operating-system/",
               "https://www.geeksforgeeks.org/installation-guide/installing-and-configuring-jenkins-on-arch-based-linux-distributions-manjaro/"),
    bs_kwargs = dict(parse_only=bs4.SoupStrainer([
        "article", "div", "section", "main", "header", "footer", "aside", "p", "span"
        ]
        )
    )
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)




prompt = PromptTemplate.from_template(
    "Use the following context to answer the question.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "Answer:"
)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


In [97]:
result = graph.invoke({"question": "How do I setup jenkins"})

# print(f"Context: {result['context']}\n\n")
print(f"Answer: {result['answer']}")

Answer: To set up Jenkins on your Linux machine, follow these steps:

1.  **Install Java Development Kit (JDK):**
    ```bash
    $ sudo pacman -S jdk8-openjdk
    ```
2.  **Change the default JRE version:**
    ```bash
    $ sudo archlinux-java set java-8-openjdk/jre
    ```
3.  **Install Jenkins from the official repository:**
    ```bash
    $ sudo pacman -S jenkins
    ```
4.  **Start Jenkins:**
    ```bash
    $ sudo systemctl start jenkins
    ```
5.  **Check Jenkins status (optional but recommended):**
    ```bash
    $ sudo systemctl status jenkins
    ```
6.  **Enable Jenkins to start on boot:**
    ```bash
    $ sudo systemctl enable jenkins
    ```
7.  **Start configuring Jenkins in your web browser:**
    *   Navigate to `http://localhost:8090` or `https://your-server-ip-address:8090`.
    *   **Unlock admin privileges** by entering the default password. You can retrieve this password using the command:
        ```bash
        $ sudo cat /var/lib/jenkins/secrets/initialAdmi